# PyCUDA-图像处理-自适应中值滤波

In [ ]:
import pycuda.autoinit
import pycuda.driver as cu
from pycuda import gpuarray
from pycuda.reduction import ReductionKernel
from pycuda.elementwise import ElementwiseKernel
import time
import numpy as np
from pycuda.compiler import SourceModule
import string
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import  animation

In [ ]:
def load_kernel_from_file(fname,kname):
    sources = None
    kernel = None
    with open(fname,encoding='utf-8') as f:
        sources = f.read() # 通过str函数转换为unicode编码
        sm = SourceModule(str(sources))
        kernel = sm.get_function(kname)
    return kernel

def load_kernel_from_string(sources,kname):
    sm = SourceModule(str(sources))
    kernel = sm.get_function(kname)
    return kernel

In [ ]:
adaptive_median_filter_kernel = load_kernel_from_file('../../datas/cuda/29-adaptive-median-filter.cu','adaptive_median_filter_kernel')

In [ ]:
rgb = cv2.cvtColor(cv2.imread('../../datas/f4.jpg'),cv2.COLOR_BGR2RGB)
rgba = cv2.cvtColor(rgb,cv2.COLOR_RGB2RGBA)
gray = cv2.cvtColor(rgb,cv2.COLOR_RGB2GRAY)

In [ ]:
rows,cols,ch = rgb.shape

In [ ]:
block = (16,16,1)
grid = (int((cols + block[0]) / block[0]),int((rows + block[1]) / block[1]))
# grid = (cols,rows,1)

In [ ]:
MAX_AREA_SIZE = 7
dst = np.zeros_like(rgb)
median = np.zeros((rows * cols * MAX_AREA_SIZE),dtype=rgb.dtype)
adaptive_median_filter_kernel(cu.In(rgb),cu.Out(dst),np.int32(cols),np.int32(rows),cu.Out(median),grid=grid,block=block)

In [ ]:
plt.imshow(dst)

In [ ]:
print(dst)